In [2]:
import tensorflow as tf

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
os.environ["NCCL_DEBUG"] = "INFO"
# # export CUDA_HOME='/usr/local/cuda-10.1'
# os.environ['CUDA_HOME'] ='/usr/local/cuda-11.5'
# os.environ['PATH'] = '/usr/local/cuda-11.5/bin:' + os.environ['PATH']
# os.environ['LIBRARY_PATH'] = '/usr/local/cuda-11.5/lib64:' + os.environ['LIBRARY_PATH']
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.5/lib64:'+ os.environ['LD_LIBRARY_PATH']
# 使用设置好的环境变量
# 加载 CIFAR-10 数据集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# 归一化图像数据
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 将目标变量转换为 one-hot 编码
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 定义一个简单的 CNN 模型
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

# 使用 MirroredStrategy 进行多 GPU 训练
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# 在策略范围内创建和编译模型
# with strategy.scope():
model = create_model()  
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
Number of devices: 8
Epoch 1/10
782/782 [==============================] - 8s 8ms/step - loss: 1.5173 - accuracy: 0.4590 - val_loss: 1.2714 - val_accuracy: 0.5551
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 1.1694 - accuracy: 0.5915 - val_loss: 1.1404 - val_accuracy: 0.6042
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 1.0358 - accuracy: 0.6375 - val_loss: 1.0926 - val_accuracy: 0.6143
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.9514 - accuracy: 0.6701 - val_loss: 0.97

In [3]:
os.environ['PATH'] ,os.environ['CUDA_HOME']

('/data4/conda_envs/g2/bin:/usr/local/cuda-11.6/bin:/home/beihang/.vscode-server/bin/0ee08df0cf4527e40edc9aa28f4b5bd38bbff2b2/bin/remote-cli:/usr/bin:/data4/conda_envs/g2/bin:/sbin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/home/beihang/anaconda3/condabin:/bin:/usr/bin:/usr/local/bin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/bin:/usr/bin:/bin:/usr/bin:/usr/local/bin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/bin:/usr/bin:/usr/local/cuda/bin:/snap/bin',
 '/usr/local/cuda-11.6')

In [4]:
import tensorflow as tf

if tf.test.is_built_with_cuda():
    print("TensorFlow can access CUDA")
else:
    print("TensorFlow cannot access CUDA")

TensorFlow can access CUDA
